In [325]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import (
    StructType,
    StructField,
    StringType,
    IntegerType,
    DoubleType,
)

In [326]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("Car_Analysis")
    .config("spark.executor.memory", "4g")
    .getOrCreate()
)

spark.sparkContext.setLogLevel("WARN")
# Tip to reader: use WARN for development, ERROR in prod

In [327]:
df = spark.read.csv(
    path="dataset/Cars Datasets 2025.csv",
    header=True,
    mode="PERMISSIVE",
)

In [328]:
df.show(1)

+-------------+-------------+-------+-------------------+----------+-----------+-------------------------+-----------+---------------+-----+------+
|Company Names|   Cars Names|Engines|CC/Battery Capacity|HorsePower|Total Speed|Performance(0 - 100 )KM/H|Cars Prices|     Fuel Types|Seats|Torque|
+-------------+-------------+-------+-------------------+----------+-----------+-------------------------+-----------+---------------+-----+------+
|      FERRARI|SF90 STRADALE|     V8|            3990 cc|    963 hp|   340 km/h|                  2.5 sec|$1,100,000 |plug in hyrbrid|    2|800 Nm|
+-------------+-------------+-------+-------------------+----------+-----------+-------------------------+-----------+---------------+-----+------+
only showing top 1 row


## Data Transformations

### Renaming Column Names to Camel Case

In [339]:
def to_camel_case(column_name):
    """
    Convert column name to camel case (Python PEP8 style)
    Example: "Company Names" -> "company_names"
    """

    name = column_name.lower()
    name = name.replace(' ', '_').replace('/', '_').replace('(', '').replace(')', '')
    name = ''.join(c if c.isalnum() or c == '_' else '' for c in name)
    name = '_'.join(filter(None, name.split('_')))
    return name

In [340]:
current_columns = df.columns
column_mapping = {col: to_camel_case(col) for col in current_columns}

for old_name, new_name in column_mapping.items():
    df = df.withColumnRenamed(old_name, new_name)

In [341]:
df = df \
     .withColumnRenamed('company_names', 'company') \
     .withColumnRenamed('cars_names', 'name') \
     .withColumnRenamed('engines', 'engine') \
     .withColumnRenamed('horsepower', 'horse_power') \
     .withColumnRenamed('total_speed', 'speed') \
     .withColumnRenamed('performance0_100_km_h', 'performance') \
     .withColumnRenamed('cars_prices', 'price') \
     .withColumnRenamed('fuel_types', 'fuel')

In [336]:
df.show(1)

+-------+-------------+------+-------------------+-----------+--------+-----------+-----------+---------------+-----+------+
|company|         name|engine|cc_battery_capacity|horse_power|   speed|performance|      price|           fuel|seats|torque|
+-------+-------------+------+-------------------+-----------+--------+-----------+-----------+---------------+-----+------+
|FERRARI|SF90 STRADALE|    V8|            3990 cc|     963 hp|340 km/h|    2.5 sec|$1,100,000 |plug in hyrbrid|    2|800 Nm|
+-------+-------------+------+-------------------+-----------+--------+-----------+-----------+---------------+-----+------+
only showing top 1 row


In [284]:
df_clean = df.dropna(how="any")

In [285]:
df_clean = (
    df_clean.withColumnRenamed("Company Names", "company_name")
    .withColumnRenamed("Cars Names", "car_name")
    .withColumnRenamed("Total Speed", "total_speed")
    .withColumnRenamed("Performance(0 - 100 )KM/H", "performance")
    .withColumnRenamed("HorsePower", "horse_power")
    .withColumnRenamed("Fuel Types", "fuel_type")
    .withColumnRenamed("Cars Prices", "price")
    .withColumnRenamed('Seats', 'seats')
)

In [286]:
df_clean = (
    df_clean.withColumn("horse_power", F.regexp_replace("horse_power", r"\(.*?\)", ""))
    .withColumn("horse_power", F.regexp_replace("horse_power", r"[^\d\-\./]", " "))
    .withColumn("horse_power", F.trim(F.col("horse_power")))
)

In [287]:
df_clean = df_clean.withColumn(
    "horse_power",
    F.when(
        F.col("horse_power").rlike(r"^\d+\s*-\s*\d+$"),
        (
            F.split(F.col("horse_power"), "-").getItem(0).cast("int")
            + F.split(F.col("horse_power"), "-").getItem(1).cast("int")
        )
        / 2,
    )
    .when(
        F.col("horse_power").rlike(r"^\d+\s*/\s*\d+$"),
        (
            F.split(F.col("horse_power"), "/").getItem(0).cast("int")
            + F.split(F.col("horse_power"), "/").getItem(1).cast("int")
        )
        / 2,
    )
    .when(
        F.col("horse_power").rlike(r"^\d+(\.\d+)?$"),
        F.col("horse_power").cast("double"),
    )
    .otherwise(None),
)

In [288]:
df_clean = (
    df_clean.withColumn("price", F.regexp_replace("price", r"[^\d]", ""))
    .withColumn("price", F.trim(F.col("price")))
    .withColumn("price", F.when(F.col("price") == "", None) # handling "" values.
                .otherwise(F.col("price").cast("double") # convert all other to double
    ))
)
df_clean.show(1)

+------------+-------------+-------+-------------------+-----------+-----------+-----------+---------+---------------+-----+------+
|company_name|     car_name|Engines|CC/Battery Capacity|horse_power|total_speed|performance|    price|      fuel_type|seats|Torque|
+------------+-------------+-------+-------------------+-----------+-----------+-----------+---------+---------------+-----+------+
|     FERRARI|SF90 STRADALE|     V8|            3990 cc|      963.0|   340 km/h|    2.5 sec|1100000.0|plug in hyrbrid|    2|800 Nm|
+------------+-------------+-------+-------------------+-----------+-----------+-----------+---------+---------------+-----+------+
only showing top 1 row


In [289]:
df_clean = (
    df_clean.withColumn(
        "total_speed", F.regexp_replace("total_speed", r"[^\d]", "")
    ) \
    .withColumn("total_speed", F.trim(F.col("total_speed")))
    .withColumn(
        "total_speed", F.when(F.col("total_speed") == "", None) # handling "" values.
        .otherwise(F.col("total_speed").cast("double")          # convert all other to double
    ))
)
df_clean.show(1)

+------------+-------------+-------+-------------------+-----------+-----------+-----------+---------+---------------+-----+------+
|company_name|     car_name|Engines|CC/Battery Capacity|horse_power|total_speed|performance|    price|      fuel_type|seats|Torque|
+------------+-------------+-------+-------------------+-----------+-----------+-----------+---------+---------------+-----+------+
|     FERRARI|SF90 STRADALE|     V8|            3990 cc|      963.0|      340.0|    2.5 sec|1100000.0|plug in hyrbrid|    2|800 Nm|
+------------+-------------+-------+-------------------+-----------+-----------+-----------+---------+---------------+-----+------+
only showing top 1 row


In [290]:
df_clean.createOrReplaceTempView("car_dataset")

### Average horse_power by Fuel type

In [147]:
avg_hs_fuelt_type = spark.sql(
    """
    select fuel_type, round(avg(horse_power), 2) as avg_hp
    from car_dataset
    group by fuel_type
    order by avg_hp desc
    """
)

### Most Powerful Engine per Company

In [148]:
most_powerful = spark.sql(
    """
    select * 
    from (
        select company_name, car_name, total_speed, 
            row_number() over(partition by company_name order by horse_power desc) as rn
        from car_dataset
    ) t
    where rn = 1
    """
)

### Fastest Acceleration Ranking - in process

### Top 3 Expensive Cars per Company

In [217]:
top3_expensive = spark.sql(
    """
    select * 
    from ( 
        select company_name, car_name, price, 
            dense_rank() over(partition by company_name order by price desc) as rank
        from car_dataset
    ) t
    where rank <= 3
    """
)
top3_expensive.show()

+------------+--------------------+---------+----+
|company_name|            car_name|    price|rank|
+------------+--------------------+---------+----+
|ASTON MARTIN|            VALKYRIE|    3.2E7|   1|
|ASTON MARTIN|              VICTOR|    1.3E7|   2|
|ASTON MARTIN|            VALHALLA|    1.1E7|   3|
|ASTON MARTIN|       LAGONDA TARAF|    1.1E7|   3|
|        AUDI|          AUDI R8 Gt|2532900.0|   1|
|        AUDI|         R8 V10 PLUS|1940000.0|   2|
|        AUDI|       RS7 SPORTBACK|1140000.0|   3|
|       Acura|          NSX Type S|1570000.0|   1|
|       Acura|NSX 3.5L Hybrid C...|1570000.0|   1|
|       Acura|  NSX Carbon Edition|1570000.0|   1|
|       Acura|NSX GT3 (Track Ve...|1500000.0|   2|
|       Acura|     TLX PMC Edition| 620000.0|   3|
|     BENTLEY|Continental GT Azure|3110000.0|   1|
|         BMW|        Mclaren 720s|4990000.0|   1|
|         BMW|         I8 ROADSTER|1650000.0|   2|
|         BMW|       M8 GRAN COUPE|1460000.0|   3|
|     Bugatti|          Centodi

### Average Speed per Seat Count - in process

### Cars Above Company Average Speed

In [296]:
company_avg_speed = spark.sql(
    """
    select company_name, round(avg(total_speed)) as avg_speed 
    from car_dataset 
    group by company_name
    """
)
company_avg_speed.show(10)

+-----------------+---------+
|     company_name|avg_speed|
+-----------------+---------+
|       Volkswagen|    204.0|
|          Peugeot|    202.0|
|         MERCEDES|    250.0|
|      LAMBORGHINI|    334.0|
|          HYUNDAI|    200.0|
|            KIA  |    200.0|
|Jaguar Land Rover|    250.0|
|             Jeep|    190.0|
|       Mitsubishi|    174.0|
|              Kia|    209.0|
+-----------------+---------+
only showing top 10 rows


In [297]:
cars_above_avg = spark.sql(
    """
    select company_name, car_name, total_speed
    from car_dataset c
    where total_speed > (
        select round(avg(total_speed))
        from car_dataset
        where company_name = c.company_name
    )
    """
)
cars_above_avg.show()

+------------+-------------------+-----------+
|company_name|           car_name|total_speed|
+------------+-------------------+-----------+
|     FERRARI|      SF90 STRADALE|      340.0|
|        AUDI|         AUDI R8 Gt|      320.0|
|         BMW|       Mclaren 720s|      341.0|
| LAMBORGHINI|    VENENO ROADSTER|      356.0|
|     FERRARI|         F8 TRIBUTO|      340.0|
|     FERRARI|            812 GTS|      340.0|
|     FERRARI|          MONZA SP2|      340.0|
|     FERRARI|          F8 SPIDER|      340.0|
|      TOYOTA|           GR SUPRA|      250.0|
|      TOYOTA|          TOYOTA 86|      226.0|
|      TOYOTA|       TOYOTA  GR86|      226.0|
|      TOYOTA|TOYOTA LAND CRUISER|      220.0|
|      TOYOTA|     TOYOTA SEQUOIA|      200.0|
|      NISSAN|               GT-R|      315.0|
|      NISSAN|               370Z|      250.0|
|      NISSAN|            Z PROTO|      290.0|
|      NISSAN|             MAXIMA|      240.0|
|ASTON MARTIN|           VALKYRIE|      402.0|
|ASTON MARTIN

### Cars Above Overall Average Horsepower

In [324]:
spark.sql(
    """
    select company_name, car_name, horse_power
    from car_dataset
    where horse_power > (select round(avg(horse_power), 2) as avg_hp
                         from car_dataset)
    order by horse_power desc
    """
).show()

+------------+--------------------+-----------+
|company_name|            car_name|horse_power|
+------------+--------------------+-----------+
|     Bugatti|              Bolide|     1850.0|
|     Bugatti|  Chiron Super Sport|     1600.0|
|     Bugatti|          Centodieci|     1600.0|
|     Bugatti|             Mistral|     1600.0|
|     Bugatti|              Chiron|     1500.0|
|     Bugatti|    Chiron Pur Sport|     1500.0|
|     Bugatti|        Chiron Sport|     1500.0|
|     Bugatti|        Chiron Noire|     1500.0|
|     Bugatti|                Divo|     1500.0|
|     Bugatti|    La Voiture Noire|     1500.0|
|ASTON MARTIN|            VALKYRIE|     1160.0|
|ASTON MARTIN|            VALHALLA|     1000.0|
|     FERRARI|       SF90 STRADALE|      963.0|
|ASTON MARTIN|              VICTOR|      836.0|
|         GMC|Hummer EV SUV Edi...|      830.0|
|         GMC|Hummer EV SUV Ext...|      830.0|
|         GMC|Hummer EV Adventu...|      830.0|
| LAMBORGHINI|                SIAN|     

### Acceleration Performance Buckets

### Most Common Fuel Type per Company

In [303]:
spark.sql(
    """
    select company_name, fuel_type, count(fuel_type)
    from car_dataset
    group by company_name, fuel_type
    order by company_name
    """
).show()

+------------+---------------+----------------+
|company_name|      fuel_type|count(fuel_type)|
+------------+---------------+----------------+
|ASTON MARTIN|         Petrol|               9|
|ASTON MARTIN|         Hybrid|               2|
|        AUDI|         Petrol|              19|
|        AUDI|       Electric|               2|
|       Acura|         Hybrid|               7|
|       Acura|         Petrol|              20|
|     BENTLEY|         Petrol|               1|
|         BMW|         Hybrid|               1|
|         BMW|         Petrol|              30|
|         BMW|         Diesel|              10|
|     Bugatti|         Petrol|              10|
|    Cadillac|         Petrol|              17|
|    Cadillac|       Electric|               3|
|   Chevrolet|       Electric|               4|
|   Chevrolet|         Diesel|               4|
|   Chevrolet|         Petrol|              50|
|     FERRARI|plug in hyrbrid|               1|
|     FERRARI|         Petrol|          

In [314]:
most_common = spark.sql(
    """
    select company_name, fuel_type, cnt,
        dense_rank() over(partition by company_name order by cnt desc) as rank
    from (
        select company_name, fuel_type, count(fuel_type) as cnt
        from car_dataset
        group by company_name, fuel_type
    ) t
    """
)

most_common.filter(F.col('rank') == 1) \
           .select(['company_name', 'fuel_type', 'cnt']) \
           .show()

+-----------------+---------+---+
|     company_name|fuel_type|cnt|
+-----------------+---------+---+
|     ASTON MARTIN|   Petrol|  9|
|             AUDI|   Petrol| 19|
|            Acura|   Petrol| 20|
|          BENTLEY|   Petrol|  1|
|              BMW|   Petrol| 30|
|          Bugatti|   Petrol| 10|
|         Cadillac|   Petrol| 17|
|        Chevrolet|   Petrol| 50|
|          FERRARI|   Petrol|  8|
|             Ford|   Petrol| 32|
|              GMC|   Petrol| 48|
|            HONDA|   Petrol|  5|
|          HYUNDAI|   Petrol| 11|
|Jaguar Land Rover|   Petrol| 34|
|             Jeep|   Petrol| 14|
|              KIA|   Petrol| 11|
|            KIA  |   Petrol|  1|
|              Kia|   Petrol| 39|
|      LAMBORGHINI|   Petrol| 23|
|         MAHINDRA|   Diesel|  2|
+-----------------+---------+---+
only showing top 20 rows


### Top Speed Leaders by Seat Category

### Company Ranking by Avg Horsepower

In [320]:
spark.sql(
    """

    select company_name, avg_hp,
        dense_rank() over(order by avg_hp desc) as rank
    from (
            select company_name, round(avg(horse_power), 2) as avg_hp
            from car_dataset
            group by company_name
    ) t
    """
).show()

+-----------------+------+----+
|     company_name|avg_hp|rank|
+-----------------+------+----+
|          Bugatti|1565.0|   1|
|          FERRARI|709.89|   2|
|     ASTON MARTIN|701.09|   3|
|      LAMBORGHINI|691.54|   4|
|     ROLLS ROYCE | 591.0|   5|
|      ROLLS ROYCE|583.76|   6|
|          BENTLEY| 550.0|   7|
|            Tesla|514.71|   8|
|          Porsche|463.26|   9|
|            Volvo|456.67|  10|
|         MERCEDES|432.86|  11|
|             AUDI|393.19|  12|
|              GMC|377.44|  13|
|         Cadillac| 368.4|  14|
|Jaguar Land Rover|355.65|  15|
|             Jeep|318.89|  16|
|           NISSAN| 316.4|  17|
|             Ford|314.97|  18|
|            Acura|311.04|  19|
|        Chevrolet|303.67|  20|
+-----------------+------+----+
only showing top 20 rows


25/08/26 10:19:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/08/26 10:19:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/08/26 10:19:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/08/26 10:19:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/08/26 10:19:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/08/26 10:19:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
